In [ ]:
import tensorflow as tf

# Tải mô hình Keras từ tệp .h5
model = tf.keras.models.load_model("C:/Users/ADM/Desktop/CAOHOC/MV/final_pj/Machine_Vision/Model/efficientnetb0.h5")
# Dòng mã trên sử dụng TensorFlow để tải mô hình đã huấn luyện từ tệp có định dạng .h5. Mô hình này được lưu trong tệp 'efficientnetb0.h5'.

# Chuyển đổi mô hình Keras sang TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# Đoạn mã trên sử dụng TFLiteConverter để chuyển đổi mô hình Keras sang mô hình TensorFlow Lite.
# Việc chuyển đổi này giúp mô hình giảm kích thước và được tối ưu để chạy trên các thiết bị di động hoặc nhúng.

# Lưu mô hình TensorFlow Lite dưới dạng tệp .tflite
with open("C:/Users/ADM/Desktop/CAOHOC/MV/final_pj/Machine_Vision/Model/efficientnetb0.tflite", "wb") as f:
    f.write(tflite_model)
# Sau khi chuyển đổi thành công, mô hình TensorFlow Lite được lưu vào tệp 'efficientnetb0.tflite'.
# Tệp này sẽ có thể được sử dụng trên các thiết bị di động hoặc các thiết bị nhúng với tài nguyên hạn chế.

# In thông báo khi quá trình chuyển đổi hoàn tất
print("Model đã được chuyển sang .tflite")
# Dòng mã trên sẽ in ra thông báo "Model đã được chuyển sang .tflite" khi quá trình chuyển đổi và lưu trữ hoàn tất.


In [ ]:
import cv2
import time
import numpy as np
from tflite_runtime.interpreter import Interpreter
from collections import Counter

# Tải mô hình phát hiện mặt sử dụng mạng SSD (Single Shot Multibox Detector)
net = cv2.dnn.readNetFromCaffe("C:/Users/ADM/Desktop/CAOHOC/MV/final_pj/Machine_Vision/Model/deploy.prototxt", 
                               "C:/Users/ADM/Desktop/CAOHOC/MV/final_pj/Machine_Vision/Model/res10_300x300_ssd_iter_140000.caffemodel")
# Đoạn mã này tải mô hình phát hiện khuôn mặt sử dụng mạng SSD đã được huấn luyện trước từ tệp "deploy.prototxt" và "res10_300x300_ssd_iter_140000.caffemodel".

# Tải mô hình TensorFlow Lite
interpreter = Interpreter(model_path="C:/Users/ADM/Desktop/CAOHOC/MV/final_pj/Machine_Vision/Model/efficientnetb0.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Mở tệp mô hình TensorFlow Lite ("efficientnetb0.tflite") và chuẩn bị các tensor đầu vào/đầu ra.

# Danh sách nhãn của mô hình nhận diện khẩu trang
labels = ["incorrect_mask", "with_mask", "without_mask"]
# Danh sách này chứa các nhãn mà mô hình dự đoán: "incorrect_mask", "with_mask" và "without_mask".

# Định nghĩa màu sắc cho từng nhãn trong không gian màu BGR
label_colors = {
    "incorrect_mask": (0, 165, 255),   # Màu vàng cho "incorrect_mask"
    "with_mask": (0, 255, 0),          # Màu xanh lá cho "with_mask"
    "without_mask": (0, 0, 255)        # Màu đỏ cho "without_mask"
}

# Mở webcam (chỉ định thiết bị webcam)
cap = cv2.VideoCapture(0)
frame_count = 0  # Đếm số khung hình
detect_interval = 10  # Thiết lập tần suất kiểm tra phát hiện khuôn mặt mỗi 10 khung hình
results = []  # Danh sách chứa kết quả dự đoán
boxes = []    # Danh sách chứa các bounding box phát hiện được

start_time = time.perf_counter()  # Lưu thời gian bắt đầu để tính FPS
fps = 0  # Khởi tạo FPS

while True:
    ret, frame = cap.read()  # Đọc một khung hình từ webcam
    if not ret:
        break  # Nếu không thể đọc được khung hình, thoát khỏi vòng lặp

    frame_count += 1
    h, w = frame.shape[:2]  # Lấy chiều cao và chiều rộng của khung hình

    # Thực hiện phát hiện khuôn mặt mỗi vài khung hình (dựa vào biến detect_interval)
    if frame_count % detect_interval < 3:
        # Chuyển đổi ảnh đầu vào thành blob để đưa vào mạng SSD
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)  # Đưa blob vào mạng SSD để phát hiện khuôn mặt
        detections = net.forward()  # Dự đoán các khuôn mặt trong ảnh
        results = []  # Reset kết quả trước mỗi lần phát hiện
        boxes = []    # Reset bounding box trước mỗi lần phát hiện

        # Duyệt qua tất cả các phát hiện mặt
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]  # Lấy độ tin cậy của phát hiện mặt
            if confidence > 0.5:  # Chỉ giữ lại phát hiện có độ tin cậy trên 50%
                # Chuyển đổi các tọa độ box từ tỷ lệ sang pixel
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                x1, y1, x2, y2 = box.astype("int")  # Lấy tọa độ của box
                x1, y1 = max(0, x1), max(0, y1)  # Đảm bảo tọa độ không vượt quá ảnh
                x2, y2 = min(w - 1, x2), min(h - 1, y2)  # Đảm bảo tọa độ không vượt quá ảnh
                face_img = frame[y1:y2, x1:x2]  # Cắt ảnh khuôn mặt

                if face_img.size == 0:
                    continue  # Nếu không có khuôn mặt, bỏ qua

                # Resize ảnh khuôn mặt và chuẩn hóa giá trị pixel về khoảng [0, 1]
                face_resized = cv2.resize(face_img, (224, 224))
                face_normalized = face_resized.astype("float32") / 255.0
                face_array = np.expand_dims(face_normalized, axis=0)

                # Dự đoán nhãn cho ảnh khuôn mặt từ mô hình TensorFlow Lite
                interpreter.set_tensor(input_details[0]['index'], face_array)  # Cấp dữ liệu vào mô hình
                interpreter.invoke()  # Chạy mô hình
                predictions = interpreter.get_tensor(output_details[0]['index'])  # Lấy kết quả dự đoán

                # Lấy nhãn và độ tin cậy của dự đoán
                class_index = int(np.argmax(predictions))  # Tìm nhãn có độ tin cậy cao nhất
                conf = predictions[0][class_index]  # Độ tin cậy của nhãn
                label = labels[class_index]  # Lấy tên nhãn
                results.append((label, conf))  # Lưu kết quả
                boxes.append((x1, y1, x2, y2, label, conf))  # Lưu bounding box

    # Vẽ bounding box và nhãn lên ảnh
    for (x1, y1, x2, y2, label, conf) in boxes:
        color = label_colors.get(label, (255, 255, 255))  # Lấy màu sắc phù hợp với nhãn
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # Vẽ bounding box
        text = f"{label} ({conf*100:.1f}%)"  # Hiển thị nhãn và độ tin cậy
        cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)  # Vẽ text lên ảnh

    # Tính toán FPS (frames per second) mỗi 10 khung hình
    if frame_count % 10 == 0:
        end_time = time.perf_counter()  # Thời gian kết thúc
        fps = 10 / (end_time - start_time)  # Tính FPS
        start_time = end_time  # Cập nhật lại thời gian bắt đầu

    # Hiển thị FPS lên ảnh
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Hiển thị khung hình
    cv2.imshow("Face Mask Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):  # Thoát khi nhấn phím 'q'
        break

# Giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()
